# 7 - Heat equation

Topics covered in this tutorial:

- [Derham](https://struphy.pages.mpcdf.de/struphy/sections/subsections/feec_derham.html#module-struphy.feec.psydac_derham) with homogeneous Dirichlet boundary conditions
- creation of a [WeightedMassOperator](https://struphy.pages.mpcdf.de/struphy/sections/subsections/feec_weightedmass.html#struphy.feec.mass.WeightedMassOperator) as diffusion matrix
- time-dependent version of [ImplicitDiffusion](https://struphy.pages.mpcdf.de/struphy/sections/subsections/propagators_fields.html#struphy.propagators.propagators_fields.ImplicitDiffusion) propagator

In what follows we present some examples of the following problem:
Let $\Omega \subset \mathbb R^d$ be open. We want to find $\phi(t) \in H^1(\Omega)$, $t \in [0, T]$, such that

$$
\begin{aligned}
\frac{\partial \phi(t, \mathbf x)}{\partial t} - \nabla \cdot \big[D(\mathbf x) \,\nabla \phi(t, \mathbf x)\big] &= 0\qquad \mathbf x \in \Omega\,,
\\[2mm]
\phi(0, \mathbf x) &= \phi_0(\mathbf x)\,,
\end{aligned}
$$

for suitable boundary conditions, where $D:\Omega \to \mathbb R^{d \times d}$ is a positive diffusion matrix, and $\phi_0\in H^1(\Omega)$ is the initial condition.

In [ ]:
from struphy.propagators.propagators_fields import ImplicitDiffusion

# default parameters of the Propagator
opts = ImplicitDiffusion.options(default=True)
opts

## 1D Gaussian blob

In [ ]:
# set up Derham complex
from struphy.feec.psydac_derham import Derham

Nel = [32, 1, 1]
p = [1, 1, 1]
spl_kind = [False, True, True]
dirichlet_bc = [[True] * 2, [False] * 2, [False] * 2]
derham = Derham(Nel, p, spl_kind, dirichlet_bc=dirichlet_bc)

In [ ]:
# set up domain Omega
from struphy.geometry.domains import Cuboid

l1 = 0.0
r1 = 10.0
domain = Cuboid(l1=l1, r1=r1)

In [ ]:
# set up mass matrices
from struphy.feec.mass import WeightedMassOperators

mass_ops = WeightedMassOperators(derham, domain)

In [ ]:
# pass simulation parameters to Propagator
ImplicitDiffusion.derham = derham
ImplicitDiffusion.domain = domain
ImplicitDiffusion.mass_ops = mass_ops

In [ ]:
# initial condition
import numpy as np

phi0_xyz = lambda x, y, z: np.exp(-((x - 5.0) ** 2) / 0.3)

In [ ]:
# pullback to the logical unit cube
phi0_logical = lambda e1, e2, e3: domain.pull(phi0_xyz, e1, e2, e3)

In [ ]:
# compute initial FE coeffs by projection
coeffs = derham.P["0"](phi0_logical)

In [ ]:
# solution field in Vh_0 subset H1
phi = derham.create_spline_function("my solution", "H1", coeffs=coeffs)

In [ ]:
# propagator parameters for heat equation
sigma_1 = 1.0
sigma_2 = 1.0
sigma_3 = 0.0

# solver options
solver = opts["solver"]
solver["recycle"] = True

# instantiate Propagator for the above quation, pass data structure (vector) of FemField
prop_heat_eq = ImplicitDiffusion(
    phi.vector, sigma_1=sigma_1, sigma_2=sigma_2, sigma_3=sigma_3, divide_by_dt=True, solver=solver
)

In [ ]:
# evalaute at logical coordinates
e1 = np.linspace(0, 1, 100)
e2 = 0.5
e3 = 0.5

# time stepping
Tend = 2.0 - 1e-6
dt = 0.1

phi_of_t = []
time = 0.0
n = 0
while time < Tend:
    n += 1

    # advance in time
    prop_heat_eq(dt)
    time += dt

    # evaluate solution and push to Omega
    phi_of_t += [phi(e1, e2, e3)]

In [ ]:
from matplotlib import pyplot as plt

# push to Omega for plotting
x, y, z = domain(e1, e2, e3, squeeze_out=True)

for funvals in phi_of_t:
    fh_xyz = domain.push(funvals, e1, e2, e3, squeeze_out=True)

    # plot
    plt.plot(x, fh_xyz)
    plt.xlabel("x")
    plt.title(f"{n} time steps")

## Polar coordinates with diffusion matrix 

Let $\Omega \subset \mathbb R^2$ to be the unit disc and assume diffusion along the isolines of the radial coordinate,

$$
 \mathbf b(x, y) = \frac{1}{\sqrt{x^2 + y^2}}
 \begin{pmatrix}
 y \\ -x
 \end{pmatrix}\,,
$$

i.e. the diffusion matrix is given by

$$
 D(x, y) = \mathbf b(x, y) \otimes \mathbf b(x, y)\,.
$$

In [ ]:
# set up Derham complex
Nel = [32, 32, 1]
p = [1, 1, 1]
spl_kind = [False, True, True]
derham = Derham(Nel, p, spl_kind)

In [ ]:
# set up domain Omega
from struphy.geometry.domains import HollowCylinder

a1 = 0.1
a2 = 4.0
Lz = 1.0
domain = HollowCylinder(a1=a1, a2=a2, Lz=Lz)

In [ ]:
# set up mass matrices
mass_ops = WeightedMassOperators(derham, domain)

In [ ]:
# pass simulation parameters to Propagator
ImplicitDiffusion.derham = derham
ImplicitDiffusion.domain = domain
ImplicitDiffusion.mass_ops = mass_ops

In [ ]:
# solution field in Vh_0 subset H1
phi = derham.create_spline_function("my solution", "H1")

In [ ]:
# initial condition
phi0_xyz = lambda x, y, z: np.exp(-((x - 2.0) ** 2) / 0.3) * np.exp(-((y) ** 2) / 0.3)

In [ ]:
# pullback to the logical unit cube
phi0_logical = lambda e1, e2, e3: domain.pull(phi0_xyz, e1, e2, e3)

In [ ]:
# evaluate initial condition in logical space
e1 = np.linspace(0, 1, 101)
e2 = np.linspace(0, 1, 101)
e3 = 0.5

funvals = phi0_logical(e1, e2, e3)

In [ ]:
# push to Omega
fh_xyz = domain.push(funvals, e1, e2, e3, squeeze_out=True)
print(f"{fh_xyz.shape = }")

x, y, z = domain(e1, e2, e3, squeeze_out=True)
print(f"{x.shape = }")

In [ ]:
# plot at z=0.5
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
ax = axs[0]

ax.contourf(x, y, fh_xyz, levels=51)
ax.axis("equal")
ax.set_title("Initial condition")
ax.set_xlabel("x")
ax.set_ylabel("y")

# add isolines of r-coordinate
for i in range(x.shape[0]):
    if i % 5 == 0:
        ax.plot(x[i], y[i], c="tab:blue", alpha=0.4, linewidth=0.5)

In [ ]:
# create diffusion matrix
bx = lambda x, y, z: y / np.sqrt(x**2 + y**2)
by = lambda x, y, z: -x / np.sqrt(x**2 + y**2)
bz = lambda x, y, z: 0.0 * x

# vector-field pullback
bv = lambda e1, e2, e3: domain.pull((bx, by, bz), e1, e2, e3, kind="v")

In [ ]:
# creation of callable Kronecker matrix
def Dmat_call(e1, e2, e3):
    bv_vals = bv(e1, e2, e3)

    # array from 2d list gives 3x3 array is in the first two indices
    tmp = np.array([[bi * bj for bj in bv_vals] for bi in bv_vals])

    # numpy operates on the last two indices with @
    return np.transpose(tmp, axes=(2, 3, 4, 0, 1))

In [ ]:
# create and assembla mass matrix
Dmat = mass_ops.create_weighted_mass("Hcurl", "Hcurl", name="bb", weights=[Dmat_call, "sqrt_g"], assemble=True)

In [ ]:
# compute initial FE coeffs by projection
phi.vector = derham.P["0"](phi0_logical)

In [ ]:
# propagator parameters for heat equation
sigma_1 = 1.0
sigma_2 = 1.0
sigma_3 = 0.0

# solver options
solver = opts["solver"]
solver["recycle"] = True

# instantiate Propagator for the above quation, pass data structure (vector) of FemField
prop_heat_eq = ImplicitDiffusion(
    phi.vector, sigma_1=sigma_1, sigma_2=sigma_2, sigma_3=sigma_3, diffusion_mat=Dmat, divide_by_dt=True, solver=solver
)

In [ ]:
# time stepping
Tend = 6.0 - 1e-6
dt = 0.1

phi_of_t = []
time = 0.0
n = 0
while time < Tend:
    n += 1

    # advance in time
    prop_heat_eq(dt)
    time += dt

    # evaluate solution and push to Omega
    phi_of_t += [phi(e1, e2, e3)]

In [ ]:
for funvals in phi_of_t:
    fh_xyz = domain.push(funvals, e1, e2, e3, squeeze_out=True)

# plot
ax_t = axs[1]
ax_t.contourf(x, y, fh_xyz, levels=51)
ax_t.axis("equal")
ax_t.set_title(f"{n} time steps")
ax_t.set_xlabel("x")
ax_t.set_ylabel("y")

# add isolines of r-coordinate
for i in range(x.shape[0]):
    if i % 5 == 0:
        ax_t.plot(x[i], y[i], c="tab:blue", alpha=0.4, linewidth=0.5)

fig